### 1. 키워드파일 업로드

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# CSV 파일에서 클러스터링 결과 로드
df_inheritance = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')

c:\Users\user\.conda\envs\nof\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. 사건명가져오기 위해서 판례목록 데이터 불러오기

In [2]:
# 사건 리스트 로드
inheritance_list = pd.read_csv('inheritance_list.csv')

# 사건명 추가를 위해 df_inheritance 데이터프레임과 병합
df_inheritance = df_inheritance.merge(inheritance_list[['사건번호', '사건명']], on='사건번호', how='left')

In [3]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166 entries, 0 to 4165
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례정보일련번호       4166 non-null   int64 
 1   사건번호           4166 non-null   object
 2   선고             4166 non-null   object
 3   판결유형           4166 non-null   object
 4   판시사항           3250 non-null   object
 5   판결요지           2932 non-null   object
 6   참조조문           3225 non-null   object
 7   참조판례           1935 non-null   object
 8   판례내용           4166 non-null   object
 9   판례내용_상단        4166 non-null   object
 10  판례내용_이유        4166 non-null   object
 11  판례내용_이유_전처리    4146 non-null   object
 12  판례내용_이유_불용어제거  4131 non-null   object
 13  클러스터           4166 non-null   int64 
 14  키워드            4166 non-null   object
 15  사건명            3924 non-null   object
dtypes: int64(2), object(14)
memory usage: 520.9+ KB


#### - 키워드, 판례내용_이유 임베딩하기

In [4]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용과 클러스터 키워드 임베딩 생성
df_inheritance['이유임베딩'] = df_inheritance['판례내용_이유'].apply(lambda x: encode_sentence(x) if pd.notnull(x) else None)
df_inheritance['키워드임베딩'] = df_inheritance['키워드'].apply(lambda x: encode_sentence(x) if pd.notnull(x) else None)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


### 3. 유사도 계산해서 추천사건명 출력하기

#### - 사용자가 입력한 키워드와 키워드임베딩, 내용임베딩 유사도 평균계산 추천하기

In [16]:
# 클러스터 주제 및 키워드 데이터 정의
data = {
    '주제': [
        '상속 및 재산 평가', '부동산 소유권 및 취득', '부동산 상속 및 사망 관련',
        '부동산 등기 및 소유권 이전', '손해배상 및 사망 보상', '농지 및 경작',
        '상속세 및 납부', '부동산 등기 및 거래', '상속 분할 및 유류분', '보험 및 보상'
    ],
    '키워드': [
        ['가액', '시가', '평가', '상속', '세법', '주식', '재산', '과세', '증여', '상속세'],
        ['점유', '토지', '취득시효', '등기', '소유권', '완성', '자주', '부동산', '취득', '의사'],
        ['신청', '상속', '부동산', '주택', '사망', '건물', '계약', '회사', '재산', '호주'],
        ['등기', '임야', '소유권', '부동산', '이전', '명의', '원인', '상속', '명의신탁', '토지'],
        ['망인', '손해', '차량', '위자료', '지급', '손해배상', '사망', '유족', '운전', '운행'],
        ['농지', '분배', '농지개혁법', '토지', '상환', '등기', '본건', '소유권', '경작', '완료'],
        ['상속세', '세액', '부과', '납부', '과세', '상속', '재산', '가액', '신고', '납세'],
        ['토지', '등기', '소유권', '이전', '명의', '분할', '지번', '주소', '환지', '매매'],
        ['상속', '재산', '분할', '유류분', '상속인', '부동산', '민법', '한정승인', '피상', '포기'],
        ['보험', '보험금', '계약', '지급', '망인', '보험료', '자동차', '연금', '수익', '상해']
    ]
}

df_clusterdata = pd.DataFrame(data)

# 키워드를 보여주는 함수
def show_keywords(topic_index):
    for button in buttons:
        button.destroy()
    buttons.clear()

    keywords = df_clusterdata.iloc[topic_index]['키워드']
    for keyword in keywords:
        button = tk.Button(root, text=keyword, command=lambda k=keyword: keyword_click(k))
        button.pack(pady=5)
        buttons.append(button)

# 키워드 클릭 시 호출되는 함수
def keyword_click(keyword):
    user_keyword = keyword
    user_keyword_embedding = encode_sentence(user_keyword)

    similarity_scores = []
    for idx, row in df_inheritance.iterrows():
        if row['키워드임베딩'] is not None and row['이유임베딩'] is not None:
            keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
            content_similarity = cosine_similarity(user_keyword_embedding, row['이유임베딩']).flatten()[0]
            average_similarity = (keyword_similarity + content_similarity) / 2
            similarity_scores.append((row['사건번호'], average_similarity))

    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_n = sorted_scores[:10]

    result = "사용자 키워드와 관련된 사건명:\n"
    for 사건번호, score in top_n:
        사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
        사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
        result += f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}\n"

    messagebox.showinfo("추천 사건명", result)

# 주제 선택 시 호출되는 함수
def topic_selected(event):
    topic_index = df_clusterdata.index[df_clusterdata['주제'] == topic_var.get()].tolist()[0]
    show_keywords(topic_index)

# 메인 윈도우 생성
root = tk.Tk()
root.title("원하는 판례찾기")

# 주제 선택 드롭다운 메뉴 생성
topic_var = tk.StringVar(value='관련 주제를 선택하세요')
topic_menu = tk.OptionMenu(root, topic_var, *df_clusterdata['주제'], command=topic_selected)
topic_menu.pack(pady=10)

# 버튼 리스트 초기화
buttons = []

# 메인 루프 실행
root.mainloop()

In [5]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 유사도 계산 및 사건명 추출
similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['키워드임베딩'] is not None and row['이유임베딩'] is not None:
        # 키워드 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
        
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['이유임베딩']).flatten()[0]
        
        # 두 유사도의 평균을 계산
        average_similarity = (keyword_similarity + content_similarity) / 2
        
        # 결과 저장
        similarity_scores.append((row['사건번호'], average_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
top_n = sorted_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명:")
for 사건번호, score in top_n:
    # 사건번호를 통해 사건명 조회
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명:
사건번호: 2010누30330, 사건명: 상속세 부과처분 취소, 유사도: 0.4304
사건번호: 2003다64381, 사건명: 종헌결의무효확인, 유사도: 0.4286
사건번호: 2007카기134, 사건명: 위헌법률심판제청, 유사도: 0.4264
사건번호: 2021나31998, 사건명: 공유물분할, 유사도: 0.4257
사건번호: 2018두32927, 사건명: 취득세등경정청구거부처분취소, 유사도: 0.4248
사건번호: 94머211, 사건명: 소유권이전등기말소청구사건, 유사도: 0.4239
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.4238
사건번호: 2005도5338, 사건명: 특정경제범죄가중처벌등에관한법률위반(횡령)(인정된죄명:업무상횡령)·업무상횡령, 유사도: 0.4237
사건번호: 88다카3847, 사건명: 토지소유권확인등, 유사도: 0.4224
사건번호: 93타경23863, 사건명: 부동산임의경매, 유사도: 0.4222


#### - 사용자가 입력한 키워드와 키워드임베딩 유사도로 추천하기

In [6]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 키워드 임베딩 유사도 계산 및 사건명 추출
keyword_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['키워드임베딩'] is not None:
        # 키워드 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
        keyword_similarity_scores.append((row['사건번호'], keyword_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_keyword_scores = sorted(keyword_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_keywords = sorted_keyword_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 (키워드 임베딩):")
for 사건번호, score in top_n_keywords:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 (키워드 임베딩):
사건번호: 2023다221144, 사건명: 예금[투자신탁 형태 단기금융집합투자기구(MMF) 수익권의 공동상속 효과가 문제된 사건], 유사도: 0.4670
사건번호: 2021두52143, 사건명: 상속세부과처분취소, 유사도: 0.4670
사건번호: 2022누67403, 사건명: 상속세부과처분취소, 유사도: 0.4670
사건번호: 2022스625, 사건명: 상속재산관리인선임, 유사도: 0.4670
사건번호: 2021다294674, 사건명: 예금반환 [망인의 공동상속인 중 1인인 원고가 은행인 피고를 상대로 망인의 청약저축예금 반환을 구하는 사안], 유사도: 0.4670
사건번호: 2019나2050091, 사건명: 상속회복청구등의소, 유사도: 0.4670
사건번호: 2019구합83052, 사건명: 상속세경정거부처분취소, 유사도: 0.4670
사건번호: 2017스516, 517, 사건명: 사건명 없음, 유사도: 0.4670
사건번호: 2016브5(본심판), 2016브6(반심판), 2016브7(반심판), 2016브8(병합), 사건명: 사건명 없음, 유사도: 0.4670
사건번호: 2015브102(본심판), 2015브103(반심판), 사건명: 사건명 없음, 유사도: 0.4670


#### - 사용자가 입력한 키워드와 내용 유사도로 추천하기

In [7]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['이유임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['이유임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 (판례내용 임베딩):")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 (판례내용 임베딩):
사건번호: 2006다46346, 사건명: 소유권이전등기등, 유사도: 0.3989
사건번호: 2010누30330, 사건명: 상속세 부과처분 취소, 유사도: 0.3938
사건번호: 81다카1061, 사건명: 부당이득반환, 유사도: 0.3917
사건번호: 2003다64381, 사건명: 종헌결의무효확인, 유사도: 0.3901
사건번호: 2017두44091, 사건명: 물납불허가처분취소청구, 유사도: 0.3869
사건번호: 2007카기134, 사건명: 위헌법률심판제청, 유사도: 0.3858
사건번호: 87누898, 사건명: 증여세등부과처분취소, 유사도: 0.3848
사건번호: 2021나31998, 사건명: 공유물분할, 유사도: 0.3843
사건번호: 91다27655, 사건명: 소유권이전등기, 유사도: 0.3841
사건번호: 68다2094, 사건명: 소유권이전등기말소등, 유사도: 0.3832
